In [1]:
# !pip install sqlalchemy
# !pip install pymysql

In [2]:
# SQL Alchemy
from sqlalchemy import create_engine

# PyMySQL 
import pymysql
pymysql.install_as_MySQLdb()

# Set path 
# import sys
# sys.path.append('../../../../')

# Config variables
from config import remote_db_endpoint, remote_db_port
from config import remote_db_name, remote_db_user, remote_db_pwd
from config import local_db_user, local_db_pwd, local_db_endpoint, local_db_port, local_db_name

# Import Pandas
import pandas as pd

In [5]:
# Create Engine and Pass in MySQL Connection

# local Postgres DB Connection  
local_engine = create_engine(f"postgresql://{local_db_user}:{local_db_pwd}@{local_db_endpoint}:{local_db_port}")

# Cloud MySQL Database Connection on AWS
engine = create_engine(f"mysql://{remote_db_user}:{remote_db_pwd}@{remote_db_endpoint}:{remote_db_port}/{remote_db_name}")

In [6]:
# Create a local database engine connection
local_conn = local_engine.connect()

# Create a remote database engine connection
conn = engine.connect()

In [7]:
# Create local gwsis database
try:
    local_conn.execution_options(isolation_level="AUTOCOMMIT").execute(f"CREATE DATABASE {local_db_name}")
except Exception as e:
    print(e)
    
# Connect to local gwsis database
try:
    local_engine = create_engine(f"postgresql://{local_db_user}:{local_db_pwd}@{local_db_endpoint}:{local_db_port}/{local_db_name}")
    local_conn = local_engine.connect()
except Exception as e:
    print(e)    

(psycopg2.errors.DuplicateDatabase) database "gwsis" already exists

[SQL: CREATE DATABASE gwsis]
(Background on this error at: http://sqlalche.me/e/f405)


## ETL Example 1

This code segment will migrate student enrollment data from your MySQL database hosted on AWS to your local Postgres database hosted on your machine.

In [8]:
# Query the remote MySQL database
student_data = pd.read_sql("SELECT * FROM all_student_enrollment", conn)
student_data.head()

,CourseName,Section,InstructorName,StudentFullName,EnrollmentStartDate,EnrollmentEndDate
0,Data Visualization and Analytics,GWU-ARL-DATA-PT-09-0,"Williams, Dartanion","Abdulrahim, Jawaher",2020-03-16,None
1,Data Visualization and Analytics,GWU-ARL-DATA-PT-09-0,"Williams, Dartanion","Clarke, Michael",2020-03-16,None
2,Data Visualization and Analytics,GWU-ARL-DATA-PT-09-0,"Williams, Dartanion","Damdinsuren, Undraa",2020-03-16,None
3,Data Visualization and Analytics,GWU-ARL-DATA-PT-09-0,"Williams, Dartanion","Doucet, Trevor",2020-03-16,None
4,Data Visualization and Analytics,GWU-ARL-DATA-PT-09-0,"Williams, Dartanion","Fingerut, Cole",2020-03-16,None


In [9]:
student_data_load = student_data 
#student_data_load.drop(columns=['ID_Student'], inplace=True)

In [10]:
student_data_load.to_sql(name='all_student_enrollment', if_exists='replace', con=local_conn, chunksize=500, index=False)

In [11]:
local_student_data = pd.read_sql("SELECT * FROM all_student_enrollment", local_conn)
print(len(local_student_data))
local_student_data.head(50)

28


,CourseName,Section,InstructorName,StudentFullName,EnrollmentStartDate,EnrollmentEndDate
0,Data Visualization and Analytics,GWU-ARL-DATA-PT-09-0,"Williams, Dartanion","Abdulrahim, Jawaher",2020-03-16,None
1,Data Visualization and Analytics,GWU-ARL-DATA-PT-09-0,"Williams, Dartanion","Clarke, Michael",2020-03-16,None
2,Data Visualization and Analytics,GWU-ARL-DATA-PT-09-0,"Williams, Dartanion","Damdinsuren, Undraa",2020-03-16,None
3,Data Visualization and Analytics,GWU-ARL-DATA-PT-09-0,"Williams, Dartanion","Doucet, Trevor",2020-03-16,None
4,Data Visualization and Analytics,GWU-ARL-DATA-PT-09-0,"Williams, Dartanion","Fingerut, Cole",2020-03-16,None
5,Data Visualization and Analytics,GWU-ARL-DATA-PT-09-0,"Williams, Dartanion","Hoff, David",2020-03-16,None
6,Data Visualization and Analytics,GWU-ARL-DATA-PT-09-0,"Williams, Dartanion","Hoffer, Katherine",2020-03-16,None
7,Data Visualization and Analytics,GWU-ARL-DATA-PT-09-0,"Williams, Dartanion","Huynh, Patrick",2020-03-16,None
8,Data Visualization and Analytics,GWU-ARL-DATA-PT-09-0,"Williams, Dartanion","Karr, Todd",2020-03-16,None
9,Data Visualization and Analytics,GWU-ARL-DATA-PT-09-0,"Williams, Dartanion","Kelly, Carly",2020-03-16,None


## ETL Example 2

This code segment will migrate data from a local csv to your MySQL database hosted on AWS

In [12]:
census_data = pd.read_csv('../Resources/Census_Data.csv')
census_data.head()

,CityState,city,state,Population,White Population,Black Population,Native American Population,Asian Population,Hispanic Population,Education None,...,Employment Female Computer Engineering,Median Age,Median Male Age,Median Female Age,Household Income,Income Per Capita,Median Gross Rent,Median Home Value,lat,lng
0,"HOUSTON, TX",HOUSTON,TX,3061887,1775897,684416,11586,230549,1368287,54180,...,22637,33.439583,32.550000,34.363542,56206.50000,32239.52083,956.708333,178233.6842,29.775734,-95.414548
1,"CHICAGO, IL",CHICAGO,IL,2702091,1318869,843633,7554,161478,785374,32800,...,18209,34.526786,33.798214,35.141071,57735.96429,38730.83929,1119.928571,264739.2857,41.867838,-87.673440
2,"BROOKLYN, NY",BROOKLYN,NY,2595259,1126111,870465,8744,297890,509243,48934,...,14845,35.175676,33.367568,36.578378,51469.18919,28309.67568,1261.783784,605743.2432,40.652805,-73.956528
3,"LOS ANGELES, CA",LOS ANGELES,CA,2426413,1068202,324842,15949,273829,1292382,62684,...,12329,35.335484,34.535484,36.061290,47494.58333,30073.19355,1201.766667,557115.0000,34.042209,-118.303468
4,"MIAMI, FL",MIAMI,FL,1820704,1361009,363514,2250,33144,1162711,27137,...,6969,38.740741,37.120370,40.262963,51232.90741,25949.35185,1260.833333,243279.6296,25.760268,-80.298510


In [13]:
census_data.to_sql(name='census_data', if_exists='append', con=conn, chunksize=500, index=False)

In [14]:
remote_census_data = pd.read_sql("SELECT * FROM census_data WHERE city='Chicago'", conn)
print(len(remote_census_data))
remote_census_data.head(50)

1


,CityState,city,state,Population,White Population,Black Population,Native American Population,Asian Population,Hispanic Population,Education None,...,Employment Female Computer Engineering,Median Age,Median Male Age,Median Female Age,Household Income,Income Per Capita,Median Gross Rent,Median Home Value,lat,lng
0,"CHICAGO, IL",CHICAGO,IL,2702091,1318869,843633,7554,161478,785374,32800,...,18209,34.526786,33.798214,35.141071,57735.96429,38730.83929,1119.928571,264739.2857,41.867838,-87.67344
